In [1]:
from Loader import fileloader as fl
from Loader import datahandledatabase as dhdb
from Loader import datacompute as dc
from Loader import datapreparation as dp
import pandas as pd
import numpy as np
from datetime import datetime
import time
import posixpath

month = '201812'
month_unitario = 201812

inifile = fl.ReadIniFile()

defaultpath = inifile.getDefaultPath()
testpath = inifile.getTestPath()

#testpath = 'D:/Datos de Usuario/cleon/Documents/Mercado Empresas/Data Fuente Comisiones/test/'
#defaultpath = 'D:/Datos de Usuario/cleon/Documents/Mercado Empresas/Data Fuente Comisiones/xlsx/'

#defaultpath = 'D:/Datos de Usuario/cleon/Documents/Mercado Empresas/Data Fuente Comisiones/xlsx/meses previos/201809/'

#*****************Actualizar periodo*************
#defaultpath = 'D:/Datos de Usuario/cleon/Documents/Mercado Empresas/Data Fuente Comisiones/xlsx/meses previos/201804/'


DEBUG:    Values loaded from globals.ini:
DEBUG:    Values loaded from globals.ini:
DEBUG:     * datapath D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel Peru S.A/MercadoEmpresas
DEBUG:     * datapath D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel Peru S.A/MercadoEmpresas
DEBUG:     * scriptspath D:/Datos de Usuario/jsaenza/Documents/Comisiones/Comisiones-python
DEBUG:     * scriptspath D:/Datos de Usuario/jsaenza/Documents/Comisiones/Comisiones-python
DEBUG:     * databasepath D:/Datos de Usuario/jsaenza/Documents/Comisiones/SQLiteDB
DEBUG:     * databasepath D:/Datos de Usuario/jsaenza/Documents/Comisiones/SQLiteDB
DEBUG:    defaultpath value is D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx
DEBUG:    defaultpath value is D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx
DEBUG:    testpath value is D:/Datos de Usuario/jsaenza/Documents/OneDrive - Entel

In [2]:
# Configurando los archivos de configuración para importar archivos tipo : xlsx, txt, ini
parser = inifile.getIniFileParser()

loader = fl.LoadFileProcess(month)
loader.setParser(parser)
loader.setDefaultPath(defaultpath)

In [3]:
# Configurando los archivos de configuración para administrar la base de datos
dbparser = inifile.getDbIniFileParser()
#inidb = fl.ReadIniFile('C:/Anaconda3/MeScripts/Comisiones4/Config/mydbconfig.ini')
#dbparser = inidb.readFile()

dbmanager = dhdb.DbDataProcess(month)
dbmanager.setParser(dbparser)


Also setting database database path to /Users/johnnysaenz/Documents/Comisiones/Mercado Empresas/SQLiteDB/mercado_empresas_db.sqlite


In [5]:
# Carga de Panel de Plataformas Comerciales: 

# Preparación de Data : Quitar formulas y quitar pestañas ocultas. Si hay intercambio de supervisores (o nuevo supervisor) 
# actualizar logins equivalentes y metricas conjuntas
# File : _Plataformas Comerciales Panel

logins = loader.loadFile('Logins')
metricasconjuntas = loader.loadFile('Metricas_conjuntas')
kpispltfrs = loader.loadFile('Kpis_plataformas')
comisionantespltfrs = loader.loadFile('Comisionantes_plataformas')
panelpltfrs0 = loader.loadHistoricalFile('Panel_plataformas')
periodo = loader.getPeriodo()

paramspltfrs= {'tipo' : 'plataformas', 'periodo' : periodo.upper(), 'keycol': 'NOMBRES', 'logins' : logins, 
               'metricasconjuntas' : metricasconjuntas, 'kpis' : kpispltfrs, 'comisionantes' : comisionantespltfrs, 
               'cuotas' : panelpltfrs0, 'resultados' : None}

panelpltfrs = loader.prepareHistoricalFiles(paramspltfrs)

panelpltfrs.to_csv(posixpath.join(testpath, month + '_panelpltfrs.csv'),encoding='utf-8-sig')

#"""

Archivo: /Users/johnnysaenz/Developer/Comisiones-python/Data/logins equivalentes.xlsx
Hoja Importada: Similar
El tamaño de Logins es 155 registros
Archivo: /Users/johnnysaenz/Developer/Comisiones-python/Data/metricas conjuntas.xlsx
Hoja Importada: reporte
El tamaño de Metricas_conjuntas es 87 registros
Archivo: /Users/johnnysaenz/OneDrive - Entel Peru S.A/MercadoEmpresas/Comisiones Plataformas Comerciales/201812_Comisiones Plataformas Comerciales.xlsx
Hoja Importada: Leyenda
El tamaño de Kpis_plataformas es 129 registros
El tamaño de Comisionantes_plataformas es 150 registros
Archivo: /Users/johnnysaenz/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx/201812_Plataformas Comerciales Panel.xlsx
Hoja Importada: Resumen Indicadores
Hoja Importada: Plataforma Carterizada
Hoja Importada: Plataforma Inbound
Hoja Importada: Plataforma Outbound
Hoja Importada: Canales Presenciales
Hoja Importada: Canales Online
El tamaño de Panel_plataformas es 10401 registros


In [5]:
#Carga de Panel de voz
#*************************
# Agregar mes enero faltante en cuotas de GC
# Asegúrate en colocar 0 en la cabecera de los resultados y eliminar toda la data fuera del formato

#"""
logins = loader.loadFile('Logins') # ojo con tener logins duplicados. Sale errores raros
metricasconjuntas = loader.loadFile('Metricas_conjuntas')
kpisvoz = loader.loadFile('Kpis_voz')
comisionantesvoz = loader.loadFile('Comisionantes_voz')
cuotasvoz = loader.loadHistoricalFile('Cuotas')

#"""

periodo = loader.getPeriodo()

#"""

#"""

# Preparar los resultados antes de enviar el cerrado
# GCE_Resultados debe tener dos o mas Hojas sino lanza error

resultadosvoz = loader.loadHistoricalFile('Resultados') # Por precaución eliminar tres columnas posteriores a dic-18.
#resultadosvoz = pd.DataFrame() # Usar si no hay resultados
                                                         
churn = loader.loadHistoricalFile('Churn')
calidad = loader.loadHistoricalFile('Calidad')

#qnp = loader.loadHistoricalFile('QNP') 
#tracking = loader.loadHistoricalFile('Tracking') # Asegurarse el nombre del kpi en resultados
#epa = loader.loadHistoricalFile('EPA') # Asegurarse el nombre del kpi en resultados

resultadosvoz = resultadosvoz.append(churn, ignore_index = True)
resultadosvoz = resultadosvoz.append(calidad, ignore_index = True)

#resultadosvoz = resultadosvoz.append(qnp, ignore_index = True)
#resultadosvoz = resultadosvoz.append(tracking, ignore_index = True)
#resultadosvoz = resultadosvoz.append(epa, ignore_index = True)

#"""

Archivo: /Users/johnnysaenz/Developer/Comisiones-python/Data/logins equivalentes.xlsx
Hoja Importada: Similar
El tamaño de Logins es 155 registros
Archivo: /Users/johnnysaenz/Developer/Comisiones-python/Data/metricas conjuntas.xlsx
Hoja Importada: reporte
El tamaño de Metricas_conjuntas es 87 registros
Archivo: /Users/johnnysaenz/OneDrive - Entel Peru S.A/MercadoEmpresas/Comisiones Grandes Cuentas/201812_Comisiones Soluciones de Negocio.xlsm
Hoja Importada: Leyenda
Archivo: /Users/johnnysaenz/OneDrive - Entel Peru S.A/MercadoEmpresas/Comisiones Grandes Cuentas/201812_Comisiones Grandes Cuentas.xlsm
Hoja Importada: Leyenda
Archivo: /Users/johnnysaenz/OneDrive - Entel Peru S.A/MercadoEmpresas/Comisiones Medianas y Pequeñas Pymes/201812_Comisiones Pymes.xlsm
Hoja Importada: Leyenda
El tamaño de Kpis_voz es 254 registros
El tamaño de Comisionantes_voz es 316 registros
Archivo: /Users/johnnysaenz/OneDrive - Entel Peru S.A/MercadoEmpresas/Data Fuente Comisiones/xlsx/201812_Formato de cuotas 

In [6]:
# Preparando la ultima data : HC, VAS

#hc = loader.loadFile('HC')
#vas = dbmanager.loadData('View_VAS_Voz')
#vas = pd.DataFrame() # temporal mientras se arregla el código

#periodo = loader.getPeriodo() # arriba se repite


"""
criteria = {'VAS' : {'colgroupby' : ['GERENCIA2' , 'ZONAVENTA', 'DEPARTAMENTO', 'VENDEDOR_CROSS_SELLING'], 
                     'colsum' : 'ACCESS_REAL'},
            'HC' : {'colfilter' : 'CARGO', 'colfilteritem' : 'CONSULTOR', 'colgroupby' : 'DATOS', 'colsum' : 'VENDEDORES'}}

otherparams = {'periodo' : periodo.upper(), 'frames' : {'HC' : hc, 'VAS' : vas}, 'criteria': criteria}
otherdfs = loader.prepareOtherPlainFiles(otherparams) 


for df in otherdfs.keys():
    resultadosvoz = resultadosvoz.append(otherdfs[df], ignore_index = True)

"""


#"""
paramsvoz = {'tipo' : 'voz', 'periodo' : periodo.upper(), 'keycol': 'LOGIN', 'logins' : logins, 
             'metricasconjuntas' : metricasconjuntas, 'kpis' : kpisvoz, 'comisionantes' : comisionantesvoz, 
             'cuotas' : cuotasvoz, 'resultados' : resultadosvoz}


panelvoz = loader.prepareHistoricalFiles(paramsvoz)          
panelvoz.to_csv(posixpath.join(testpath, month + '_panelvoz.csv'),encoding='utf-8-sig')
#"""

In [6]:
cuotasvoz.to_csv(testpath + 'cuotas_brutas.csv')